In [1]:
!tiffinfo MSB-06824-08-05.svs

=== TIFF directory 0 ===
TIFF Directory at offset 0x3ec9be (4114878)
  Subfile Type: (0 = 0x0)
  Image Width: 5975 Image Length: 5321 Image Depth: 1
  Tile Width: 240 Tile Length: 240
  Bits/Sample: 8
  Compression Scheme: JPEG
  Photometric Interpretation: RGB color
  YCbCr Subsampling: 2, 2
  Samples/Pixel: 3
  Planar Configuration: single image plane
  ImageDescription: Aperio Image Library v12.0.15 
6096x5421 [0,100 5975x5321] (240x240) JPEG/RGB Q=70|AppMag = 40|StripeWidth = 2032|ScanScope ID = SS75592|Filename = 229293|Date = 08/01/24|Time = 09:52:52|Time Zone = GMT-04:00|User = 329fd652-d7b7-4339-90a6-3ad5938221e4|MPP = 0.2523|Left = 30.677172|Top = 11.292421|LineCameraSkew = -0.000609|LineAreaXOffset = -0.002272|LineAreaYOffset = -0.002321|Focus Offset = 0.000000|DSR ID = aperio06|ImageID = 229293|Exposure Time = 45|Exposure Scale = 0.000001|DisplayColor = 0|SessonMode = NR|OriginalWidth = 6096|OriginalHeight = 5421|ICC Profile = AT2
  ICC Profile: <present>, 1687824 bytes
  JP

In [2]:
import openslide  # OpenSlide: WSI(Whole Slide Image)를 열기 위한 라이브러리
import os  # 디렉토리 및 파일 경로 관련 작업을 위한 모듈
from PIL import Image  # 이미지 처리용 라이브러리 (Pillow)
import numpy as np  # 배열 연산을 위한 NumPy
from tqdm import tqdm  # 진행 상황 표시용 프로그레스바 라이브러리

In [3]:
# 패치의 크기 (256x256 픽셀)
patch_size = 256 
stride = 256  # 패치 간의 간격 (겹치지 않도록 stride = patch_size)
level = 0  # OpenSlide에서 사용할 해상도 레벨 (0이 가장 높은 해상도)
output_dir = "patches_filtered"  # 결과 패치를 저장할 디렉토리 이름
image_name = "MSB-06824-08-05.svs"

In [4]:
# OpenSlide를 사용해 WSI 파일 열기 
# Open WSI slide using OpenSlide
slide = openslide.OpenSlide(image_name)
width, height = slide.level_dimensions[level]  # 해당 레벨에서의 전체 슬라이드 너비와 높이

# 출력 디렉토리가 없다면 생성  
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# 패치에 조직(tissue)이 포함되어 있는지 판단하는 함수
# Function to check if a patch contains tissue (not mostly background)
def is_tissue(patch, threshold=0.8):
    
    # 패치를 RGB 배열로 변환 
    #Convert patch to RGB numpy array
    np_patch = np.array(patch.convert("RGB")) 
    
    # 평균값을 이용해 grayscale 이미지 생성
    # Convert to grayscale
    gray = np_patch.mean(axis=2)   

    # 밝은 픽셀(배경)의 비율 계산
    # Calculate bright (background) pixel ratio
    bg_ratio = (gray > 220).sum() / (gray.shape[0] * gray.shape[1])  

    # 배경 비율이 threshold보다 작으면 조직으로 간주
    # Return True if mostly tissue
    return bg_ratio < threshold   


# 저장할 패치 이미지 이름에 사용할 ID 초기화 
# Initialize patch ID
patch_id = 0  

# 슬라이드를 위에서 아래로, 왼쪽에서 오른쪽으로 순차적으로 스캔 
# Loop over slide from top to bottom and left to right
for y in tqdm(range(0, height - patch_size + 1, stride)):  # 세로 방향 반복 / Vertical steps
    for x in range(0, width - patch_size + 1, stride):  # 가로 방향 반복 / Horizontal steps
        
        # 해당 위치에서 패치 추출 
        # Extract patch from (x, y) at specified level
        patch = slide.read_region((x, y), level, (patch_size, patch_size))

        # 조직이 포함되어 있으면 저장 
        # Save only if patch contains tissue
        if is_tissue(patch):
            
            # RGBA에서 RGB로 변환 
            # Convert RGBA to RGB
            patch = patch.convert("RGB")  

            # 파일 저장 
            # Save patch image
            patch.save(os.path.join(output_dir, f"patch_{patch_id}.png")) 
            
            # 다음 패치 ID 증가  
            # Increment patch ID
            patch_id += 1  

100%|███████████████████████████████████████████| 20/20 [00:03<00:00,  6.07it/s]
